# Proyecto Shark Attack

## DATA CLEANING

In [89]:
import pandas as pd

In [90]:
#Exportamos la base de dato y creamos un duplicado sobre el que vamos a trabajar
data = pd.read_csv('GSAF2.csv')
data1 = data

In [91]:
# Comprobamos la cantidad de valores nulos por columnas
null_cols = data1.isnull().sum()
null_cols[null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [92]:
# Eliminamos las columnas que no vamos a utilizar
data1 = data1.drop(['Unnamed: 22','Unnamed: 23','Case Number','Case Number.1','Name','Case Number.2','Investigator or Source','pdf','href formula','href','original order','Injury','Type','Species '], axis=1)

In [93]:
#Renombramos algunos nombres de columnas y las ordenamos
data1 = data1.rename(index=str, columns={"Sex ": "Sex", "Time": "Hour", "Date": "Month"})
column_order = ['Year','Month','Hour','Country','Area','Location','Sex','Age','Activity','Fatal (Y/N)']
data1 = data1[column_order]
data1.head(3)




,Year,Month,Hour,Country,Area,Location,Sex,Age,Activity,Fatal (Y/N)
0,2016,18-Sep-16,13h00,USA,Florida,"New Smyrna Beach, Volusia County",M,16,Surfing,N
1,2016,18-Sep-16,11h00,USA,Florida,"New Smyrna Beach, Volusia County",M,36,Surfing,N
2,2016,18-Sep-16,10h43,USA,Florida,"New Smyrna Beach, Volusia County",M,43,Surfing,N


In [94]:
#Eliminamos todos los datos anteriores a 1900
years = data1["Year"]
drop_cols = list(years[years < 1900].index)
data1 = data1.drop(drop_cols, axis=0)

# Ordenamos la tabla por año
data1.sort_values(['Year'], ascending = 0).head()

,Year,Month,Hour,Country,Area,Location,Sex,Age,Activity,Fatal (Y/N)
0,2016,18-Sep-16,13h00,USA,Florida,"New Smyrna Beach, Volusia County",M,16,Surfing,N
52,2016,05-Jun-16,11h30,AUSTRALIA,Western Australia,Mindarie,F,60,Diving,Y
76,2016,31-Mar-16,11h00,USA,Hawaii,"Olowalu, Maui",F,46,Snorkeling,N
75,2016,07-Apr-16,NaN,USA,Florida,"Corners Beach, Jupiter, Palm Beach County",M,NaN,SUP,N
74,2016,07-Apr-16,NaN,USA,Florida,"Florida Keys, Monroe County",M,34,Fishing,N


In [95]:
null_cols1 = data1.isnull().sum()
null_cols1[null_cols1 > 0]

Hour           2633
Country          22
Area            295
Location        363
Sex             505
Age            2103
Activity        426
Fatal (Y/N)       9
dtype: int64

In [96]:
# Eliminamos las filas que tenían algún valor de las columnas Country, Fatal, Area o Location nulos
data1 = data1.dropna(subset=['Country','Fatal (Y/N)','Area','Location'], axis=0)


In [97]:
null_cols1 = data1.isnull().sum()
null_cols1[null_cols1 > 0]

Hour        2200
Sex          421
Age         1770
Activity     352
dtype: int64

In [98]:
#Cambiamos los datos nulos a Unknokn
data1[['Activity','Sex','Age','Hour']] = data1[['Activity','Sex','Age','Hour']].fillna('Unknown')

In [99]:
#Comprobamos que no queda ningún nulo
null_cols1 = data1.isnull().sum()
null_cols1[null_cols1 > 0]

Series([], dtype: int64)

In [100]:
#Nos quedamos solo con el mes del ataque 
def fecha(*lab):
    for x in lab:
        data1.loc[data1["Month"].str.contains(x), 'Month'] = x
fecha('Jan','Mar','Feb','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')

def april(*lab):
    for x in lab:
        data1.loc[data1["Month"].str.contains(x), 'Month'] = 'Apr'
april('Ap')

def nomon(*lab):
    for x in lab:
        data1.loc[data1["Month"].str.startswith(x), 'Month'] = 'Unknown'
nomon('1','2','S','R','C','F','W','E','L')

datavalue = data1[["Month"]].stack().value_counts().index.tolist()
data1["Month"].value_counts()

Unknown    970
Jul        525
Aug        488
Jan        421
Jun        385
Oct        357
Apr        353
Dec        352
Mar        330
Nov        319
May        295
Name: Month, dtype: int64

In [101]:
datavalue = data1[["Hour"]].stack().value_counts().index.tolist()
display(len(datavalue))

data1["Hour"] = data1["Hour"].str.lower()
def time(*lab):
    for x in lab:
        data1.loc[data1["Hour"].str.startswith(x), 'Hour'] = x + "h"
time('00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19',"20","21","22","23")

letras =['<','>','2 ','8','3','� ','500','"',' ','-','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
def notime(lab):
    for x in lab:
        data1.loc[data1["Hour"].str.startswith(x), "Hour"] = 'Unknown'
notime(letras)

datavalue = data1[["Hour"]].stack().value_counts().index.tolist()
data1["Hour"].value_counts().head()




334

Unknown    2675
11h         228
16h         224
14h         222
15h         218
Name: Hour, dtype: int64

In [102]:
# Corregimos el espacio que había delante de TONGA
data1['Country'] = data1['Country'].str.upper()
data1.loc[data1['Country'].str.startswith(" T"), 'Country'] = 'TONGA'
datavalue = data1[["Country"]].stack().value_counts().index.tolist()
display(len(datavalue))
data1['Country'].value_counts().head()


129

USA                 1958
AUSTRALIA           1094
SOUTH AFRICA         527
PAPUA NEW GUINEA     113
BRAZIL                93
Name: Country, dtype: int64

In [103]:
#Cambiamos algunos de los valores de la columna Sex
datavalue = data1[["Sex"]].stack().value_counts().index.tolist()
display(datavalue)

def nosex(*lab):
    for x in lab:
        data1.loc[data1['Sex'].str.startswith(x), 'Sex'] = 'Unknown'
nosex('.', 'lli', 'N')

def msex(*lab):
    for x in lab:
        data1.loc[data1['Sex'].str.startswith(x), 'Sex'] = 'M'
msex('M ')

datavalue = data1[["Sex"]].stack().value_counts().index.tolist()

data1["Sex"].value_counts()

['M', 'F', 'Unknown', 'M ', '.', 'N', 'lli']

M          3847
F           524
Unknown     424
Name: Sex, dtype: int64

In [104]:
edu = []
def edad(lab):
    for x in lab:
        data1.loc[data1["Age"].str.startswith(x), "Age"] = str(x)
    for y in range(10,90):
        edu.append(str(y))
edad(edu)

#for (i, elem) in enumerate(data1["Age"]):
    #print(i, elem)
    
#No he conseguido automatizar que me sacara el index, ni cambiar los valores de esos index por Unknown y los he tenido que extraer a mano y eliminar
ind = [386,695,1197,1262,1327,1411,1490,1491,1564,1614,2050,2548,2564,2677,2911,2968,2984,3030,3134,3271,3503,3515,3652,3804,3853,3894,4012,4041,4129,4141,4379,4460,4688,4777]
data1 = data1.drop(data1.index[ind])

def no(*lab):
    for x in lab:
        data1.loc[data1["Age"].str.contains(x), "Age"] = "Unknown"
no('young','A.M.','teen','Teen')

datavalue = data1[["Age"]].stack().value_counts().index.tolist()
display(len(datavalue))

data1["Age"].value_counts().head()

111

Unknown    1778
17          137
18          134
19          127
16          127
Name: Age, dtype: int64

In [105]:
datavalue = data1[["Activity"]].stack().value_counts().index.tolist()
display(len(datavalue))

#Vamos a reducir de 1098 actividades a categorizarlas en 21 categorías
data1['Activity'] = data1['Activity'].str.lower()

def fishing(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Fishing'
fishing('fish','lobs','catch','wade','harp','fly f','spear','hunt')

def surfing(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Surfing'
surfing('surf','"riding')

def swiming(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Swiming'
swiming('swim','tread','float','bath','"swim"','wadi','"swim')

def kayaking(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Kayaking'
kayaking('kaya','canoe')

def diving(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Diving'
diving('scub','free','div','skin','pearl')

def kite(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Kite'
kite('kite')

def snorkeling(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Snorkeling'
snorkeling('snor')

def sailing(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Sailing'
sailing('boat','sail')

def air(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Air Disaster'
air('argentin','air','boeing','"flying')

def body(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Bodyboarding'
body('boogie','body')

def play(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Playing'
play('play')

def paddling(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Paddling'
paddling('padd')

def jump(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Jumping'
jump('jump')

def shark(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Shark Fishing'
shark('shark','remov','kill')

def stand(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Standing'
stand('stand')

def unk(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Unknown'
unk('no ','.',' ')

def kite(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Kitesurfing'
kite('kite')

def sea(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Sea Disaster'
sea('sea d','ferry','3','2','1','4','9')

def wind(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Windsurfing'
wind('wind')

def helping(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Helping'
helping('help')

lower =['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
def photo(lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Other'
photo(lower)



datavalue = data1[["Activity"]].stack().value_counts().index.tolist()
data1["Activity"].value_counts().head(10)

1112

Swiming         1120
Surfing          972
Fishing          847
Other            841
Diving           373
Bodyboarding     166
Standing         117
Snorkeling        72
Kayaking          57
Paddling          28
Name: Activity, dtype: int64

In [106]:
# Unificamos los valores de la columna Fatal a Y/N/UNKNOWN
datavalue = data1[["Fatal (Y/N)"]].stack().value_counts().index.tolist()

def N(*lab):
    for x in lab:
        data1.loc[data1['Fatal (Y/N)'].str.startswith(x), 'Fatal (Y/N)'] = 'N'
N(' N','N ')

def Unkf(*lab):
    for x in lab:
        data1.loc[data1['Fatal (Y/N)'].str.startswith(x), 'Fatal (Y/N)'] = 'UNKNOWN'
Unkf('F')

datavalue = data1[["Fatal (Y/N)"]].stack().value_counts().index.tolist()

data1["Fatal (Y/N)"].value_counts()




N          3698
Y           997
UNKNOWN      66
Name: Fatal (Y/N), dtype: int64

In [107]:
ausdata = data1[(data1['Country']=='AUSTRALIA')]

## RESULTADOS ESTUDIO DE LOS ATAQUES DE TIBURONES EN AUSTRALIA DESDE EL AÑO 1900 AL 2016

### En Australia se producen el el 22,83% de los ataques de tiburones a nivel mundial

In [108]:
data1.Country.value_counts().head(3)/4761*100

USA             40.831758
AUSTRALIA       22.831338
SOUTH AFRICA    11.027095
Name: Country, dtype: float64

### En los últimos 10 años se han producido 231 ataques de tiburón, de los cuales, 28 fueron mortales.

In [109]:
display(ausdata.Year.value_counts().sort_index(ascending = False).head(10))
ausdaya = data1[(data1['Country']=='AUSTRALIA')&(data1['Year']>=2007)]
ausdaya["Fatal (Y/N)"].value_counts()








2016    22
2015    30
2014    29
2013    20
2012    26
2011    20
2010    18
2009    27
2008    20
2007    19
Name: Year, dtype: int64

N          202
Y           28
UNKNOWN      1
Name: Fatal (Y/N), dtype: int64

### Los meses donde más ataque se producen son Enero y Diciembre entre las 09:00h y las 12:00h.

In [110]:
display(ausdaya.Month.value_counts().head(3))
ausdaya.Hour.value_counts().head()

Unknown    36
Jan        31
Dec        29
Name: Month, dtype: int64

Unknown    74
11h        16
10h        16
12h        14
09h        14
Name: Hour, dtype: int64

### El 71,36% de los ataques se producen surfeando, pescando o nadando, siendoel surf la actividad con más ataques registrados en los últimos 10 años

In [111]:
ausdaya.Activity.value_counts()/231*100

Surfing         42.857143
Fishing         18.181818
Swiming         10.389610
Other            8.658009
Diving           5.627706
Bodyboarding     5.194805
Snorkeling       3.896104
Kayaking         3.030303
Paddling         1.298701
Windsurfing      0.432900
Kite             0.432900
Name: Activity, dtype: float64

### El número de ataques mientras se realiza surf ha aumentado un 135% comparando los registros desde 1900 y los de los últimos 10 años, lo que probablemente suponga un aumento de la popularidad del surf en Australia en los últimos años.

In [112]:
ausdata.Activity.value_counts()/1087*100

Swiming          20.883165
Other            20.239190
Fishing          19.227231
Surfing          18.215271
Diving           11.591536
Bodyboarding      2.299908
Snorkeling        1.747930
Standing          1.563937
Kayaking          1.287948
Paddling          0.643974
Shark Fishing     0.551978
Sea Disaster      0.551978
Sailing           0.367985
Windsurfing       0.275989
Unknown           0.275989
Playing           0.091996
Jumping           0.091996
Kite              0.091996
Name: Activity, dtype: float64

### Las regiones con mayor número de ataques en los últimos 10 años son: New South Wales, Western Australia y Queensland. Podemos estimar que son los lugares más populares para hacer surf, pescar o con buenas playas para bañarse en Australia.

In [113]:
area = ausdaya.groupby(['Country','Area']).size()
area = area.drop(area[area < 20].index)
area

Country    Area             
AUSTRALIA  New South Wales      105
           Queensland            33
           Western Australia     55
dtype: int64

### Solo el 12,27% de los ataques se producen a mujeres 

In [114]:
ausdaya.Sex.value_counts()

M          193
F           27
Unknown     11
Name: Sex, dtype: int64

## CSV

In [115]:
data.to_csv('sharkattack.csv', index=False)